exchange_rate project

In [1]:
import pandas as pd
import numpy as np

# load data

In [3]:
df = pd.read_csv("../data/stock_forex.csv")

In [18]:
df.shape

(28106, 47)

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,date,title,category,reportor,content,stock_name,stock_reference,stock_price,...,MA0201 台灣-貨幣總計數-M1B-期底-月-台幣(百萬)(TWD),MA0202 台灣M1B月底值-年增率(百分比),MA1701 台灣-貨幣總計數-M1B-日平均-月-台幣(百萬)(TWD),MA1702 台灣M1B日平均年增率(百分比),MA1801 台灣-貨幣總計數-M1A-日平均-月-台幣(百萬)(TWD),MA1901 台灣-M1A年增率-日平均值-月-月(百分比),MA2001 台灣-M1A年增率-期底值-月-月(百分比),MA4602 台灣-貨幣供給額M1B變動因素分析-國外資產淨額-月-台幣(百萬)(TWD),MA46 台灣-貨幣供給額M2變動因素分析-國外資產淨額-月-台幣(百萬)(TWD),ex_rate
0,0,0,2019-02-01,牧德進入Q1設備業出貨淡季 1月營收2.06億元月減22.13%,台股新聞,鉅亨網記者張欽發 台北,設備廠牧德科技 (3563-TW) 今 (1) 日公布 1 月營收為 2.06 億元，月減 ...,牧德(TWS:3563:STOCK),相關個股 click,NaN,...,18074220,5.97,18095040,5.97,6963974,4.39,4.55,46497,46497,30.785
1,1,1,2019-02-01,郭台銘證實 鴻海定位將從大量生產轉向研究發展,台股新聞,鉅亨網記者彭昱文 台北,鴻海 (2317-TW) 今 (1) 日召開內部員工大會，對於外界關心美國威州投資案，郭台銘...,鴻海(TWS:2317:STOCK),相關個股 click,NaN,...,18074220,5.97,18095040,5.97,6963974,4.39,4.55,46497,46497,30.785
2,2,2,2019-02-01,〈車市〉LEXUS總代理賣贏賓士 YARIS奪單一車種銷售冠軍 和泰車成大贏家,台股新聞,鉅亨網記者王莞甯 台北,元月汽車銷售統計今 (1) 日出爐，新車總掛牌數達 44143 台，月增 5.1%，但仍較去...,和泰車(TWS:2207:STOCK),相關個股 click,NaN,...,18074220,5.97,18095040,5.97,6963974,4.39,4.55,46497,46497,30.785
3,3,3,2019-02-01,永豐餘投控延攬蔡維力擔任總經理 看好其跨業戰力,台股新聞,鉅亨網記者陳蕙綾台北,永豐餘投資控股 (1907-TW) 今 (1) 日召開董事會，通過延攬前三陽工業策略長、南陽...,永豐餘(TWS:1907:STOCK),相關個股 click,NaN,...,18074220,5.97,18095040,5.97,6963974,4.39,4.55,46497,46497,30.785
4,4,4,2019-02-01,勞動基金元月收益1126億元創單月歷史新高 打消去年虧損,台股新聞,鉅亨網記者郭幸宜 台北,勞動部勞動基金運用局今 (1) 日表示，去年受全球股市劇烈震盪影響，勞動基金虧損 726 億...,None,None,None,...,18074220,5.97,18095040,5.97,6963974,4.39,4.55,46497,46497,30.785


# NLP processing 

## data preprocessing

In [6]:
# concat title and content 
# label encoding on category
# label encoding on reporter (use including "reporter's name")

In [15]:
df['all_content'] = df['title'] + df['content']

In [2]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [14]:
new_category = preprocessing.LabelEncoder()
new_category.fit(df['category'])
df['label_category'] = new_category.transform(df['category'])

In [16]:
reportor = preprocessing.LabelEncoder()
reportor.fit(df['reportor'])
df['label_reportor'] = reportor.transform(df['reportor'])

In [49]:
# load stopword
stopwords = []
with open('../data/stopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)
stopwords.append(' ')
stopwords.append('\n')

In [97]:
import jieba

In [47]:
df['result_cut'] = df['all_content'].apply(lambda sec:jieba.lcut(sec))

In [56]:
clean_result = []
for i in df['result_cut'] :
    clean_result.append(list(filter(lambda sec: sec not in stopwords,i)))

In [60]:
df['clean_result'] = clean_result

In [62]:
df.to_csv('../data/preprocess_data.csv')

In [31]:
df = pd.read_csv('../../data/crawler_exchange/preprocess_data.csv')

In [32]:
def transfrom_str_list(b) :
    
    b = b.replace('[','')
    b = b.replace(']','')
    b = b.replace("'",'')
    b = b.replace(" ",'')
    c = b.split(",")
    
    return c

In [33]:
d = df['clean_result'].apply(lambda x :transfrom_str_list(x))
df['clean_result'] = d

## create word_list

In [38]:
convert_content = []
df['clean_result'].apply(lambda x:convert_content.append(' '.join(x)))

0        None
1        None
2        None
3        None
4        None
         ... 
28101    None
28102    None
28103    None
28104    None
28105    None
Name: clean_result, Length: 28106, dtype: object

In [80]:
vectorizer = TfidfVectorizer(token_pattern=r'(?u)\b\w+\b',min_df=5)
tfidf = vectorizer.fit(convert_content)

In [104]:
a =pd.DataFrame(tfidf.idf_,tfidf.get_feature_names()).to_dict()

In [95]:
tfidf.transform(convert_content).toarray().shape

(28106, 52056)

In [81]:
weight = tfidf.transform(convert_content).toarray()
words = tfidf.vocabulary_

In [89]:
df.shape

(28106, 50)

In [88]:
weight.shape

(28106, 52056)

In [100]:
from jieba import analyse

In [116]:
jieba.analyse.TFIDF(idf_path=None) #新建 TFIDF 實例,不設置就採用默認方式 
jieba.analyse.extract_tags(convert_content[0], topK=100, withWeight=True, allowPOS=())

[('牧德', 0.6232343247957346),
 ('營收', 0.2832883294526066),
 ('億元', 0.2266306635620853),
 ('年增', 0.2266306635620853),
 ('成長', 0.2266306635620853),
 ('2019', 0.2266306635620853),
 ('市場', 0.2266306635620853),
 ('出貨', 0.16997299767156399),
 ('2018', 0.16997299767156399),
 ('新產業', 0.16997299767156399),
 ('設備業', 0.11331533178104265),
 ('2.06', 0.11331533178104265),
 ('22.13%', 0.11331533178104265),
 ('13', 0.11331533178104265),
 ('說明', 0.11331533178104265),
 ('xa0', 0.11331533178104265),
 ('中美貿易', 0.11331533178104265),
 ('確定', 0.11331533178104265),
 ('橫向', 0.11331533178104265),
 ('年營', 0.11331533178104265),
 ('季稅後純益', 0.11331533178104265),
 ('純益', 0.11331533178104265),
 ('科技', 0.08292333687312795),
 ('淡季', 0.08264354598748816),
 ('展望', 0.07348065740341232),
 ('指出', 0.07224818665023697),
 ('拓展', 0.06843434097810427),
 ('主管', 0.06815312362578199),
 ('深度', 0.06594839079772512),
 ('延伸', 0.06461091213156399),
 ('每股', 0.06067539679033176),
 ('以外', 0.05918357877251185),
 ('新高', 0.0588749435492891),


In [70]:
columns=count_vect.get_feature_names()

In [60]:
tfidf_vect = TfidfVectorizer(min_df=5)
tfidf_vect.fit(convert_content)

TfidfVectorizer(min_df=5)

In [62]:
tfidf_vect

{'牧德': 33713,
 '進入': 44886,
 'q1': 5721,
 '設備業': 41844,
 '出貨': 12834,
 '淡季': 32370,
 '營收': 33449,
 '06': 41,
 '元月': 11544,
 '22': 1072,
 '13': 545,
 '設備': 41840,
 '廠牧德': 23020,
 '科技': 36399,
 '3563': 1780,
 'tw': 6356,
 '公布': 12317,
 '億元': 11260,
 '月減': 29059,
 '年增': 22583,
 '14': 624,
 '去年': 14605,
 '亮麗': 8977,
 '2018': 982,
 '全年': 12043,
 '31': 1581,
 '創歷史': 13546,
 '新高': 27706,
 '18': 852,
 '預計': 48644,
 '日召': 27866,
 '開法': 46872,
 '說明': 42182,
 '營運': 33512,
 '展望': 21726,
 '第一季': 36982,
 '指出': 25635,
 '客戶': 20140,
 '應春節': 24359,
 '假期': 10694,
 '將裝機': 21073,
 '時程': 28334,
 '調整': 42225,
 'xa0': 6569,
 '衰退': 41315,
 '去年同期': 14607,
 '成長': 24590,
 '公司': 12299,
 '上半年': 7121,
 '整體訂': 27213,
 '單能': 16836,
 '見度': 41506,
 '逐漸': 44562,
 '明朗': 28159,
 '將維持': 21018,
 '順暢': 48560,
 '應有': 24365,
 '穩定': 36701,
 '表現': 41300,
 '2019': 983,
 '全球': 12074,
 '景氣': 28434,
 '深受': 32421,
 '中美貿易': 8288,
 '大戰': 18758,
 '影響': 23552,
 '市場': 22213,
 '確定': 36164,
 '因素': 17266,
 '增多': 18145,
 '主管': 8450,
 '萬全': 40

In [ ]:
weight = tfidf_vect
pd.DataFrame(weight,columns=word)

In [25]:
wordlist = []
for text in df['clean_result']:
    wordlist.extend(text)

In [26]:
import codecs
with codecs.open('../data/鉅亨台股2019_2020.txt', 'w',"utf-8") as f:
    for item in wordlist:
        f.write("%s\n" % item)

In [27]:
from gensim.models import word2vec


# Settings
seed = 112
sg = 1
window_size = 15
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('../data/鉅亨台股2019_2020.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('鉅亨台股2019_2020.model')

In [31]:
model.wv.similar_by_word('牧德',topn=20)

[('以累積', 0.42426052689552307),
 ('切騰訊', 0.41687822341918945),
 ('銀經驗', 0.3909519910812378),
 ('剛開春', 0.3896898925304413),
 ('insurance', 0.37912648916244507),
 ('聯嘉光', 0.3785717487335205),
 ('TOWN', 0.37266063690185547),
 ('美玲', 0.3717298209667206),
 ('首獎', 0.36320292949676514),
 ('富盛', 0.36242803931236267),
 ('富喬移', 0.3607634902000427),
 ('維護網', 0.35969278216362),
 ('郭台銘親', 0.3584022521972656),
 ('場堪', 0.35752981901168823),
 ('單留', 0.3537650406360626),
 ('1.80%', 0.35353249311447144),
 ('7967', 0.34981250762939453),
 ('1304', 0.3494550883769989),
 ('雙方的後續', 0.3489800691604614),
 ('意見', 0.34863752126693726)]

In [30]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,title,category,reportor,content,stock_name,stock_reference,...,MA1901 台灣-M1A年增率-日平均值-月-月(百分比),MA2001 台灣-M1A年增率-期底值-月-月(百分比),MA4602 台灣-貨幣供給額M1B變動因素分析-國外資產淨額-月-台幣(百萬)(TWD),MA46 台灣-貨幣供給額M2變動因素分析-國外資產淨額-月-台幣(百萬)(TWD),ex_rate,label_category,all_content,label_reportor,result_cut,clean_result
0,0,0,0,2019-02-01,牧德進入Q1設備業出貨淡季 1月營收2.06億元月減22.13%,台股新聞,鉅亨網記者張欽發 台北,設備廠牧德科技 (3563-TW) 今 (1) 日公布 1 月營收為 2.06 億元，月減 ...,牧德(TWS:3563:STOCK),相關個股 click,...,4.39,4.55,46497,46497,30.785,2,牧德進入Q1設備業出貨淡季 1月營收2.06億元月減22.13%設備廠牧德科技 (3563-...,35,"['牧德', '進入', 'Q1', '設備業', '出貨', '淡季', ' ', '1'...","[牧德, 進入, Q1, 設備業, 出貨, 淡季, 1, 月, 營收, 2.06, 億, 元..."
1,1,1,1,2019-02-01,郭台銘證實 鴻海定位將從大量生產轉向研究發展,台股新聞,鉅亨網記者彭昱文 台北,鴻海 (2317-TW) 今 (1) 日召開內部員工大會，對於外界關心美國威州投資案，郭台銘...,鴻海(TWS:2317:STOCK),相關個股 click,...,4.39,4.55,46497,46497,30.785,2,郭台銘證實 鴻海定位將從大量生產轉向研究發展鴻海 (2317-TW) 今 (1) 日召開內部...,39,"['郭台銘', '證實', ' ', '鴻海', '定位', '將從', '大量', '生產...","[郭台銘, 證實, 鴻海, 定位, 將從, 大量, 生產轉, 研究, 發展鴻海, 2317,..."
2,2,2,2,2019-02-01,〈車市〉LEXUS總代理賣贏賓士 YARIS奪單一車種銷售冠軍 和泰車成大贏家,台股新聞,鉅亨網記者王莞甯 台北,元月汽車銷售統計今 (1) 日出爐，新車總掛牌數達 44143 台，月增 5.1%，但仍較去...,和泰車(TWS:2207:STOCK),相關個股 click,...,4.39,4.55,46497,46497,30.785,2,〈車市〉LEXUS總代理賣贏賓士 YARIS奪單一車種銷售冠軍 和泰車成大贏家元月汽車銷售統...,66,"['〈', '車市', '〉', 'LEXUS', '總', '代理', '賣', '贏賓士...","[〈, 車市, 〉, LEXUS, 總, 代理, 賣, 贏賓士, YARIS, 奪單, 一車..."
3,3,3,3,2019-02-01,永豐餘投控延攬蔡維力擔任總經理 看好其跨業戰力,台股新聞,鉅亨網記者陳蕙綾台北,永豐餘投資控股 (1907-TW) 今 (1) 日召開董事會，通過延攬前三陽工業策略長、南陽...,永豐餘(TWS:1907:STOCK),相關個股 click,...,4.39,4.55,46497,46497,30.785,2,永豐餘投控延攬蔡維力擔任總經理 看好其跨業戰力永豐餘投資控股 (1907-TW) 今 (1)...,81,"['永豐餘', '投控', '延攬', '蔡維力', '擔任', '總經理', ' ', '...","[永豐餘, 投控, 延攬, 蔡維力, 擔任, 總經理, 看好, 跨業, 戰力, 永豐餘, 投..."
4,4,4,4,2019-02-01,勞動基金元月收益1126億元創單月歷史新高 打消去年虧損,台股新聞,鉅亨網記者郭幸宜 台北,勞動部勞動基金運用局今 (1) 日表示，去年受全球股市劇烈震盪影響，勞動基金虧損 726 億...,None,None,...,4.39,4.55,46497,46497,30.785,2,勞動基金元月收益1126億元創單月歷史新高 打消去年虧損勞動部勞動基金運用局今 (1) 日表...,72,"['勞動', '基金', '元月', '收益', '1126', '億元', '創單', '...","[勞動, 基金, 元月, 收益, 1126, 億元, 創單, 月, 歷史, 新高, 打消, ..."
